In [ ]:
# LABOR RIGIDITY & POST-GENAI RECOVERY (HEALTHCARE)
# FULL PIPELINE FROM RAW PBJ


import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols



returns = pd.read_csv("ai_event_study_sector_specific_market_adjusted.csv")
returns = returns[returns["sector"] == "Healthcare"].copy()


PBJ_DIR = "/Users/medharavi/Downloads/Payroll Based Journal Daily Nurse Staffing"
csv_files = glob.glob(os.path.join(PBJ_DIR, "**", "*.csv"), recursive=True)

print(f"Found {len(csv_files)} PBJ files")

def want_col(col):
    c = col.lower().replace(" ", "_")
    return (
        c == "cy_qtr"
        or c.startswith("hrs_")
        or "census" in c
    )

pbj_quarters = []

for f in csv_files:
    try:
        year = f.split("_CY")[1][:4]
        if year not in {"2019","2020","2021","2022","2023","2024"}:
            continue
    except Exception:
        continue

    df = pd.read_csv(
        f,
        encoding="latin1",
        usecols=want_col,
        low_memory=False
    )

    # Normalize columns
    df.columns = (
        df.columns
        .str.lower()
        .str.strip()
        .str.replace(" ", "_")
    )

    # Identify census
    census_cols = [c for c in df.columns if "census" in c]
    if not census_cols:
        continue
    census_col = census_cols[0]

    # Licensed staff: RN, LPN, DON
    licensed_cols = [
        c for c in df.columns
        if c.startswith("hrs_rn")
        or c.startswith("hrs_lpn")
        or "don" in c
    ]

    # All staffing hours
    hour_cols = [c for c in df.columns if c.startswith("hrs_")]
    if not licensed_cols or not hour_cols:
        continue

    # Convert numeric
    for c in hour_cols + [census_col]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # Construct row-level totals
    df["licensed_hours"] = df[licensed_cols].sum(axis=1)
    df["total_hours"] = df[hour_cols].sum(axis=1)

    # Aggregate to quarter
    q = (
        df.groupby("cy_qtr")
          .agg(
              licensed_hours=("licensed_hours", "sum"),
              total_hours=("total_hours", "sum"),
              total_census=(census_col, "sum")
          )
          .reset_index()
    )

    pbj_quarters.append(q)

# ============================================================
# 3. COMBINE QUARTERS & BUILD LABOR RIGIDITY
# ============================================================

pbj = (
    pd.concat(pbj_quarters)
      .groupby("cy_qtr")
      .sum()
      .reset_index()
)

pbj["labor_rigidity"] = pbj["licensed_hours"] / pbj["total_hours"]
pbj["event_quarter"] = pd.PeriodIndex(pbj["cy_qtr"], freq="Q").to_timestamp()

print(pbj.head())


pbj = pbj.sort_values("event_quarter")
returns = returns.sort_values("event_date")
returns["event_date"] = pd.to_datetime(returns["event_date"])

hc = pd.merge_asof(
    returns,
    pbj[["event_quarter", "labor_rigidity"]],
    left_on="event_date",
    right_on="event_quarter",
    direction="backward"
)

hc = hc.dropna(subset=["labor_rigidity"])


threshold = hc["labor_rigidity"].median()
hc["high_rigidity"] = (hc["labor_rigidity"] > threshold).astype(int)


POST_WINDOW = (0, 126)

car_panel = (
    hc[hc["days_from_event"].between(*POST_WINDOW)]
    .groupby(["event_date", "high_rigidity", "days_from_event"])["market_adj_return"]
    .mean()
    .groupby(["event_date", "high_rigidity"])
    .cumsum()
    .reset_index()
)


plt.figure(figsize=(9,5))

for grp, label in [(0, "Low Labor Rigidity"), (1, "High Labor Rigidity")]:
    tmp = car_panel[car_panel["high_rigidity"] == grp]
    mean_car = tmp.groupby("days_from_event")["market_adj_return"].mean()
    plt.plot(mean_car.index, mean_car.values, linewidth=2, label=label)

plt.axhline(0, color="gray")
plt.xlabel("Days Since GenAI Release")
plt.ylabel("Cumulative Abnormal Return")
plt.title("Healthcare Post-GenAI Recovery by Labor Rigidity")
plt.legend()
plt.tight_layout()
plt.show()


final_car = (
    hc[hc["days_from_event"].between(*POST_WINDOW)]
    .groupby(["event_date", "high_rigidity"])["market_adj_return"]
    .sum()
    .reset_index()
)

reg = ols(
    "market_adj_return ~ high_rigidity",
    data=final_car
).fit(cov_type="HC3")

print(reg.summary())


Found 24 PBJ files
Saved → pbj_quarterly_labor_intensity.csv
   cy_qtr   total_hours  total_census  hours_per_patient
0  2020Q1  7.024424e+08      93031627           7.550577
1  2021Q1  7.618355e+08      97059299           7.849176
2  2021Q2  7.544993e+08     100691670           7.493165
3  2021Q3  7.546682e+08     104304954           7.235210
4  2021Q4  7.420130e+08     102675347           7.226788
